coding: utf8

In [ ]:
from atlassian import Jira


<br>
    That example use for normalizing the permission schemes between<br>
    BROWSE_PROJECTS and WORK_ON_ISSUES parameters<br>



 Login part
<br>
jira = Jira(url="JIRA_URL", username="ATLASSIAN_LOGIN", password="ATLASSIAN_PASSWORD")<br>
et all permission schemes ID's


In [ ]:
permission_schemes = jira.get_all_permissionschemes()
scheme_id_list = []
for item in permission_schemes:
    # Get id's (except default)
    if item.get("id") != 0:
        scheme_id_list.append(item.get("id"))


Make a func for checking permissions matching
<br>
def check_if_permissions_match(permission_id):<br>
    browse_projects = []<br>
    work_on_issues = []<br>
    scheme = jira.get_permissionscheme(permission_id, expand="permissions")<br>
    for permission in scheme["permissions"]:<br>
        permission["holder"].pop("expand", None)<br>
        if permission["permission"] == "WORK_ON_ISSUES":<br>
            work_on_issues.append(permission["holder"])<br>
        elif permission["permission"] == "BROWSE_PROJECTS":<br>
            browse_projects.append(permission["holder"])<br>
    if browse_projects == work_on_issues:<br>
        missing_permissions = False<br>
    elif not work_on_issues:<br>
        missing_permissions = browse_projects<br>
    else:<br>
        missing_permissions = []<br>
        for parameter in browse_projects:<br>
            if parameter in work_on_issues:<br>
                pass<br>
            else:<br>
                missing_permissions.append(parameter)<br>
    return missing_permissions<br>
evaluate_flag = True<br>
for id in scheme_id_list:<br>
    result = check_if_permissions_match(id)<br>
    print(result)<br>
    if result and evaluate_flag:<br>
        for item in result:<br>
            grant = dict(holder=item, permission="WORK_ON_ISSUES")<br>
            jira.set_permissionscheme_grant(id, grant)